In [ ]:
#SCRIPT 1 - Demonstrasi Hukum Pertama Newton (Hukum Inersia) dengan Tkinter
# -----------------------------
# ⚙️ Demonstrasi Hukum Newton 1
# -----------------------------
import tkinter as tk
import random
from vector import Vector

class NewtonFirstLawDemo:
    def __init__(self, root):
        self.root = root
        self.root.title("Demonstrasi Hukum Pertama Newton")
        
        # Ukuran canvas
        self.width = 800
        self.height = 400
        self.canvas = tk.Canvas(root, width=self.width, height=self.height, bg='white')
        self.canvas.pack(pady=20)

        # Penjelasan teks
        explanation = """HUKUM PERTAMA NEWTON (HUKUM INERSIA):
Sebuah benda akan mempertahankan keadaan geraknya 
(tetap diam atau bergerak lurus beraturan) 
kecuali ada gaya eksternal yang bekerja padanya"""
        self.canvas.create_text(self.width//2, 30, text=explanation, font=('Arial', 12), justify='center')

        # Tombol kontrol
        self.start_btn = tk.Button(root, text="Mulai Demo", command=self.start_demo)
        self.start_btn.pack()
        self.reset_btn = tk.Button(root, text="Reset", command=self.reset)
        self.reset_btn.pack()

        # Slider gaya gesek
        self.friction_label = tk.Label(root, text="Gaya Gesek: 0")
        self.friction_label.pack()
        self.friction_slider = tk.Scale(root, from_=0, to=0.5, resolution=0.01, orient='horizontal', command=self.set_friction)
        self.friction_slider.pack(fill='x', padx=20)

        # Slider kecepatan awal
        self.velocity_label = tk.Label(root, text="Kecepatan Awal: 5")
        self.velocity_label.pack()
        self.velocity_slider = tk.Scale(root, from_=1, to=20, orient='horizontal', command=self.set_velocity)
        self.velocity_slider.pack(fill='x', padx=20)

        # Inisialisasi variabel
        self.ball = None
        self.ball_moving = False
        self.velocity = 5
        self.friction = 0
    
    def set_friction(self, value):
        self.friction = float(value)
        self.friction_label.config(text=f"Gaya Gesek: {self.friction:.2f}")
    
    def set_velocity(self, value):
        self.velocity = int(value)
        self.velocity_label.config(text=f"Kecepatan Awal: {self.velocity}")
    
    def start_demo(self):
        if not self.ball_moving:
            self.create_ball()
            self.ball_moving = True
            self.move_ball()

    def create_ball(self):
        if self.ball:
            self.canvas.delete(self.ball)
        start_y = random.randint(80, self.height - 50)

        # Posisi dan kecepatan pakai Vector
        self.pos = Vector(50, start_y)
        self.vel = Vector(self.velocity, 0)

        radius = 20
        self.radius = radius
        self.ball = self.canvas.create_oval(
            self.pos.x - radius, self.pos.y - radius,
            self.pos.x + radius, self.pos.y + radius,
            fill='blue'
        )
    
    def move_ball(self):
        if self.ball_moving:
            print(f"\n📍 Posisi: {self.pos}, Kecepatan: {self.vel}, Gesekan: {self.friction}")

            # Tambahkan kecepatan ke posisi
            self.pos.add(self.vel)

            # Update posisi bola di canvas
            self.canvas.coords(
                self.ball,
                self.pos.x - self.radius, self.pos.y - self.radius,
                self.pos.x + self.radius, self.pos.y + self.radius
            )

            # Terapkan gesekan jika ada
            if self.friction > 0:
                self.vel.mult(1 - self.friction)
                if self.vel.mag() < 0.01:
                    self.vel = Vector(0, 0)

            # Jika bola keluar dari sisi kanan, kembali ke kiri
            if self.pos.x > self.width + self.radius:
                self.pos.x = -self.radius
            elif self.pos.x < -self.radius:
                self.pos.x = self.width + self.radius

            # Lanjut animasi jika masih ada gerakan
            if self.vel.mag() > 0:
                self.root.after(50, self.move_ball)
            else:
                self.ball_moving = False
    
    def reset(self):
        self.ball_moving = False
        if self.ball:
            self.canvas.delete(self.ball)
            self.ball = None


# Jalankan program
if __name__ == "__main__":
    root = tk.Tk()
    app = NewtonFirstLawDemo(root)
    root.mainloop()

In [ ]:
#SCRIPT 2 - Demonstrasi Hukum Ketiga Newton (Aksi-Reaksi) dengan Tkinter
import tkinter as tk
from vector import Vector

class NewtonThirdLawDemo:
    def __init__(self, root):
        self.root = root
        self.root.title("Demonstrasi Hukum Ketiga Newton")

        # Inisialisasi canvas
        self.width = 800
        self.height = 400
        self.canvas = tk.Canvas(root, width=self.width, height=self.height, bg='white')
        self.canvas.pack(pady=20)

        # Tambahkan teks penjelasan
        explanation = """HUKUM KETIGA NEWTON (AKSI-REAKSI):
Untuk setiap gaya aksi, selalu ada gaya reaksi 
yang sama besar tetapi berlawanan arah"""
        self.canvas.create_text(self.width//2, 30, text=explanation, font=('Arial', 12), justify='center')

        # Buat objek roket dan bumi
        self.rocket = self.canvas.create_rectangle(100, 200, 200, 300, fill='red', tags='rocket')
        self.exhaust = self.canvas.create_oval(90, 300, 110, 310, fill='yellow', state='hidden', tags='exhaust')
        self.earth = self.canvas.create_oval(400, 200, 600, 400, fill='blue', tags='earth')

        # Posisi dan kecepatan roket menggunakan vektor
        self.rocket_pos = Vector(150, 250)
        self.rocket_vel = Vector(0, 0)
        self.earth_pos = Vector(500, 300)  # Posisi pusat bumi

        # Parameter gaya
        self.thrust = 0.5     # Gaya dorong dari roket
        self.gravity = 0.1    # Konstanta gravitasi simulasi
        self.fuel = 100       # Bahan bakar

        # Tombol kontrol
        self.start_btn = tk.Button(root, text="Mulai Mesin", command=self.start_engine)
        self.start_btn.pack(side='left', padx=10)
        self.reset_btn = tk.Button(root, text="Reset", command=self.reset)
        self.reset_btn.pack(side='left', padx=10)

        # Label bahan bakar
        self.fuel_label = tk.Label(root, text="Bahan Bakar: 100%")
        self.fuel_label.pack(side='left', padx=10)

        # Status simulasi
        self.simulating = False
        self.root.after(100, self.update)

    def start_engine(self):
        # Nyalakan mesin roket jika masih ada bahan bakar
        if self.fuel > 0:
            self.simulating = True
            self.canvas.itemconfig('exhaust', state='normal')

    def reset(self):
        # Reset posisi, kecepatan, bahan bakar
        self.simulating = False
        self.canvas.coords('rocket', 100, 200, 200, 300)
        self.canvas.itemconfig('exhaust', state='hidden')
        self.rocket_pos = Vector(150, 250)
        self.rocket_vel = Vector(0, 0)
        self.fuel = 100
        self.fuel_label.config(text="Bahan Bakar: 100%")

    def update(self):
        if self.simulating and self.fuel > 0:
            # AKSI: roket mendorong gas ke bawah → gaya ke atas
            force_up = Vector(0, -self.thrust)

            # Terapkan gaya reaksi ke roket (menambah kecepatan ke atas)
            self.rocket_vel.add(force_up)

            # Kurangi bahan bakar
            self.fuel -= 0.5
            self.fuel_label.config(text=f"Bahan Bakar: {max(0, int(self.fuel))}%")

            # Gaya gravitasi antara bumi dan roket (arah ke bumi)
            direction = Vector(self.earth_pos.x - self.rocket_pos.x,
                               self.earth_pos.y - self.rocket_pos.y)
            distance = max(50, direction.mag())  # Hindari jarak terlalu kecil
            direction.normalize()

            # Hitung besar gaya gravitasi dan ubah menjadi vektor
            gravity_strength = self.gravity * 1000 / (distance**2)
            gravity_force = direction.copy()
            gravity_force.mult(gravity_strength)

            # Terapkan gaya gravitasi ke roket
            self.rocket_vel.add(gravity_force)

            # Update posisi roket berdasarkan kecepatannya
            self.rocket_pos.add(self.rocket_vel)

            # Update posisi di canvas (menggunakan canvas.coords)
            x, y = self.rocket_pos.x, self.rocket_pos.y
            self.canvas.coords('rocket', x-50, y-50, x+50, y+50)
            self.canvas.coords('exhaust', x-10, y+50, x+10, y+60)

            # Jika bahan bakar habis, sembunyikan api knalpot
            if self.fuel <= 0:
                self.canvas.itemconfig('exhaust', state='hidden')

        # Jalankan fungsi ini terus-menerus setiap 50 ms
        self.root.after(50, self.update)

# Jalankan program
if __name__ == "__main__":
    root = tk.Tk()
    app = NewtonThirdLawDemo(root)
    root.mainloop()

In [5]:
#SCRIPT 3 - Demonstrasi Hukum Kedua Newton (F = m * a) dengan Simulasi
from vector import Vector

# === CLASS MOVER UNTUK BENDA ===
class Mover:
    def __init__(self, mass, x, y):
        self.mass = mass
        self.location = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)

    def applyForce(self, force):
        # F = m * a → a = F / m
        f = force.copy()
        f.div(self.mass)
        self.acceleration.add(f)

    def update(self):
        self.velocity.add(self.acceleration)
        self.location.add(self.velocity)
        self.acceleration.mult(0)  # reset after update

# === INISIALISASI ===
mass = 1  # bola biru
mover = Mover(mass=mass, x=100, y=200)
wind = Vector(0.1, 0)  # gaya angin

# === SIMULASI 10 LANGKAH ===
print("Langkah | Gaya (F) | Percepatan (a) | Kecepatan (v) | Posisi (x)")
print("--------|----------|----------------|----------------|----------------")

for step in range(11):
    # Tampilkan data sebelum update
    print(f"{step:>6} | "
          f"({wind.x:.2f}) | "
          f"({wind.x / mover.mass:.2f})        | "
          f"({mover.velocity.x:.2f})        | "
          f"({mover.location.x:.2f})")

    # Terapkan gaya dan update posisi
    mover.applyForce(wind)
    mover.update()


Langkah | Gaya (F) | Percepatan (a) | Kecepatan (v) | Posisi (x)
--------|----------|----------------|----------------|----------------
     0 | (0.10) | (0.10)        | (0.00)        | (100.00)
     1 | (0.10) | (0.10)        | (0.10)        | (100.10)
     2 | (0.10) | (0.10)        | (0.20)        | (100.30)
     3 | (0.10) | (0.10)        | (0.30)        | (100.60)
     4 | (0.10) | (0.10)        | (0.40)        | (101.00)
     5 | (0.10) | (0.10)        | (0.50)        | (101.50)
     6 | (0.10) | (0.10)        | (0.60)        | (102.10)
     7 | (0.10) | (0.10)        | (0.70)        | (102.80)
     8 | (0.10) | (0.10)        | (0.80)        | (103.60)
     9 | (0.10) | (0.10)        | (0.90)        | (104.50)
    10 | (0.10) | (0.10)        | (1.00)        | (105.50)


In [ ]:
#SCRIPT 4 - Demonstrasi Hukum Kedua Newton (F = m * a) dengan Tkinter
import tkinter as tk
from vector import Vector  # Pastikan kamu punya Vector class dengan .add(), .div(), .mult(), .copy()

# === CLASS UNTUK BENDA BERGERAK (MOVER) ===
class Mover:
    def __init__(self, mass, x, y, canvas, color):
        self.mass = mass
        self.location = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.radius = mass * 10
        self.canvas = canvas
        self.color = color

        # Gambar objek bola sekali saja, lalu simpan ID-nya
        r = self.radius
        self.id = canvas.create_oval(x - r, y - r, x + r, y + r, fill=color)

    def applyForce(self, force):
        # F = m * a → a = F / m
        f = force.copy()
        f.div(self.mass)
        self.acceleration.add(f)

    def update(self):
        self.velocity.add(self.acceleration)
        self.location.add(self.velocity)
        self.acceleration.mult(0)

        # Update posisi di canvas dengan canvas.coords
        x, y = self.location.x, self.location.y
        r = self.radius
        self.canvas.coords(self.id, x - r, y - r, x + r, y + r)

# === CLASS SIMULASI UTAMA ===
class Simulation:
    def __init__(self, root):
        self.root = root
        self.width = 600
        self.height = 400
        self.canvas = tk.Canvas(root, width=self.width, height=self.height, bg="white")
        self.canvas.pack()

        # Gaya angin (konstan ke kanan)
        self.wind = Vector(0.1, 0)

        # Inisialisasi dua mover dengan massa berbeda
        self.mover1 = Mover(mass=1, x=100, y=150, canvas=self.canvas, color="blue")
        self.mover2 = Mover(mass=4, x=100, y=250, canvas=self.canvas, color="red")

        # Label untuk info teks perhitungan
        self.info_text = tk.Label(root, font=("Courier", 12), justify="left", bg="white", anchor="w")
        self.info_text.pack(fill="both")

        self.update()

    def update(self):
        # Terapkan gaya ke kedua mover
        self.mover1.applyForce(self.wind)
        self.mover2.applyForce(self.wind)

        # Update posisi mereka
        self.mover1.update()
        self.mover2.update()

        # Hitung percepatan masing-masing untuk ditampilkan
        acc1 = self.wind.copy()
        acc1.div(self.mover1.mass)
        acc2 = self.wind.copy()
        acc2.div(self.mover2.mass)

        # Update teks info
        info = (
            f"Gaya angin (F): ({self.wind.x:.2f}, {self.wind.y:.2f})\n"
            f"Mover Biru (massa={self.mover1.mass}): a = F/m = {acc1.x:.2f}\n"
            f"Mover Merah (massa={self.mover2.mass}): a = F/m = {acc2.x:.2f}\n\n"
            f"Kecepatan Biru: v = ({self.mover1.velocity.x:.2f}, {self.mover1.velocity.y:.2f})\n"
            f"Kecepatan Merah: v = ({self.mover2.velocity.x:.2f}, {self.mover2.velocity.y:.2f})"
        )
        self.info_text.config(text=info)

        # Lanjutkan animasi
        self.root.after(20, self.update)

# === JALANKAN PROGRAM ===
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Simulasi Hukum Newton II (F = m * a)")
    app = Simulation(root)
    root.mainloop()


In [9]:
# SCRIPT 5 - Dua Gaya pada Benda (Simulasi Hukum Kedua Newton)
from vector import Vector

# === CLASS MOVER (BENDA) ===
class Mover:
    def __init__(self, mass, x, y):
        self.mass = mass
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.total_force = Vector(0, 0)

    def applyForce(self, force):
        # Gaya dijumlahkan dulu, belum diubah jadi percepatan
        self.total_force.add(force)

    def update(self):
        # 1. Hitung percepatan dari total gaya
        acc = self.total_force.copy()
        acc.div(self.mass)
        self.acceleration = acc

        # 2. Tambah percepatan ke kecepatan
        self.velocity.add(self.acceleration)

        # 3. Tambah kecepatan ke posisi
        self.position.add(self.velocity)

        # 4. Reset gaya total setelah update
        self.total_force = Vector(0, 0)

# === SIMULASI 10 LANGKAH ===
mover = Mover(mass=2, x=0, y=0)

# Gaya tetap
gravity = Vector(0, 0.1)  # ke bawah
wind = Vector(0.1, 0)     # ke kanan

print("Langkah | Posisi (x,y)    | Kecepatan (x,y) | Percepatan (x,y)")
print("-" * 60)

for step in range(1, 11):
    # Tambahkan gaya-gaya ke benda
    mover.applyForce(gravity)
    mover.applyForce(wind)

    # Update posisi benda
    mover.update()

    # Cetak hasil
    print(f"{step:>6} | {mover.position} | {mover.velocity} | {mover.acceleration}")


Langkah | Posisi (x,y)    | Kecepatan (x,y) | Percepatan (x,y)
------------------------------------------------------------
     1 | (0.05, 0.05) | (0.05, 0.05) | (0.05, 0.05)
     2 | (0.15, 0.15) | (0.10, 0.10) | (0.05, 0.05)
     3 | (0.30, 0.30) | (0.15, 0.15) | (0.05, 0.05)
     4 | (0.50, 0.50) | (0.20, 0.20) | (0.05, 0.05)
     5 | (0.75, 0.75) | (0.25, 0.25) | (0.05, 0.05)
     6 | (1.05, 1.05) | (0.30, 0.30) | (0.05, 0.05)
     7 | (1.40, 1.40) | (0.35, 0.35) | (0.05, 0.05)
     8 | (1.80, 1.80) | (0.40, 0.40) | (0.05, 0.05)
     9 | (2.25, 2.25) | (0.45, 0.45) | (0.05, 0.05)
    10 | (2.75, 2.75) | (0.50, 0.50) | (0.05, 0.05)


In [ ]:
#SCRIPT 6 - Simulasi Balon Helium dengan Fisika dan Koordinat
import tkinter as tk
import math
from vector import Vector

class Balloon:
    def __init__(self, mass, x, y):
        self.mass = mass
        self.location = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.radius = mass * 15
        self.forces = []

    def applyForce(self, force):
        self.forces.append(force.copy())
        f = force.copy()
        f.div(self.mass)
        self.acceleration.add(f)

    def update(self):
        self.velocity.add(self.acceleration)
        self.location.add(self.velocity)
        self.acceleration.mult(0)
        self.forces.clear()

    def checkEdges(self, height):
        if self.location.y - self.radius < 0:
            self.location.y = self.radius
            self.velocity.y *= -0.7

class Simulation:
    def __init__(self, root):
        self.root = root
        self.width = 400
        self.height = 600
        self.canvas = tk.Canvas(root, width=self.width, height=self.height, bg="skyblue")
        self.canvas.pack()

        self.balloon = Balloon(mass=1, x=self.width//2, y=self.height-50)
        self.balloon_id = None
        self.coord_text_id = None
        self.frame_count = 0
        self.info_text = None

        self.setup_info_panel()
        self.create_balloon()
        self.update()

    def setup_info_panel(self):
        info_frame = tk.Frame(self.root)
        info_frame.pack(fill=tk.X, padx=10, pady=5)

        tk.Label(info_frame, text="Variabel Fisika:", font=('Arial', 10, 'bold')).pack(anchor='w')
        self.info_text = tk.Text(info_frame, height=10, width=50, font=('Courier', 9))
        self.info_text.pack()

        rumus = (
            "Rumus:\n"
            "1. Hukum II Newton: F = m × a\n"
            "2. Percepatan: a = F_total / m\n"
            "3. Kecepatan: v = v + a × Δt\n"
            "4. Posisi: pos = pos + v × Δt\n"
        )
        self.info_text.insert(tk.END, rumus)
        self.info_text.config(state=tk.DISABLED)

    def create_balloon(self):
        """Membuat objek balon pertama kali"""
        x, y, r = self.balloon.location.x, self.balloon.location.y, self.balloon.radius
        self.balloon_id = self.canvas.create_oval(
            x - r, y - r, x + r, y + r,
            fill="pink", outline="red", width=2
        )
        self.coord_text_id = self.canvas.create_text(
            x + r + 10, y, anchor="w", font=('Courier', 10),
            text=f"x={x:.1f}, y={y:.1f}"
        )

    def update_physics_info(self):
        self.info_text.config(state=tk.NORMAL)
        self.info_text.delete(1.0, tk.END)

        helium_force = Vector(0, -0.15)
        wind_strength = math.sin(self.frame_count * 0.05) * 0.1
        wind_force = Vector(wind_strength, 0)

        total_force = Vector(helium_force.x + wind_force.x,
                             helium_force.y + wind_force.y)
        acceleration = Vector(total_force.x / self.balloon.mass,
                              total_force.y / self.balloon.mass)

        self.info_text.insert(tk.END, "=== PERHITUNGAN FRAME ===\n\n")
        self.info_text.insert(tk.END, f"1. Gaya Helium: {helium_force}\n")
        self.info_text.insert(tk.END, f"2. Gaya Angin: {wind_force}\n")
        self.info_text.insert(tk.END, f"   (wind_strength = sin({self.frame_count}×0.05)×0.1 = {wind_strength:.3f})\n")
        self.info_text.insert(tk.END, f"\n3. Total Gaya (F): {total_force}\n")
        self.info_text.insert(tk.END, f"4. Percepatan (a = F/m): {acceleration}\n")
        self.info_text.insert(tk.END, f"\n5. Kecepatan (v): {self.balloon.velocity}\n")
        self.info_text.insert(tk.END, f"6. Posisi: {self.balloon.location} (x={self.balloon.location.x:.1f}, y={self.balloon.location.y:.1f})\n")

        if self.balloon.location.y - self.balloon.radius < 0:
            self.info_text.insert(tk.END, "\n7. BALON MENABRAK ATAS!\n")
            self.info_text.insert(tk.END, f"   Kecepatan setelah pantul: {self.balloon.velocity}\n")

        self.info_text.config(state=tk.DISABLED)

    def update(self):
        # 1. Hitung gaya
        helium_force = Vector(0, -0.15)
        wind_strength = math.sin(self.frame_count * 0.05) * 0.1
        wind_force = Vector(wind_strength, 0)
        # 2. Terapkan gaya
        self.balloon.applyForce(helium_force)
        self.balloon.applyForce(wind_force)
        # 3. Update posisi dan kecepatan
        self.balloon.update()
        self.balloon.checkEdges(self.height)
        # 4. Update posisi objek di canvas
        x, y, r = self.balloon.location.x, self.balloon.location.y, self.balloon.radius
        self.canvas.coords(self.balloon_id, x - r, y - r, x + r, y + r)
        self.canvas.coords(self.coord_text_id, x + r + 10, y)
        self.canvas.itemconfig(self.coord_text_id, text=f"x={x:.1f}, y={y:.1f}")
        # 5. Update info panel
        self.update_physics_info()
        # 6. Loop
        self.frame_count += 1
        self.root.after(50, self.update)

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Simulasi Balon Helium dengan Fisika & Koordinat (coords)")
    sim = Simulation(root)
    root.mainloop()



In [17]:
# SCRIPT 7 - Simulasi Massa dan Gaya dengan Tkinter
import tkinter as tk
from vector import Vector
# -----------------------------
# Class Mover untuk benda bergerak
# -----------------------------
class Mover:
    def __init__(self, canvas, mass, x, y, color):
        self.canvas = canvas
        self.mass = mass  # Massa benda (kg)
        self.location = Vector(x, y)  # Posisi awal
        self.velocity = Vector(0, 0)  # Kecepatan awal
        self.acceleration = Vector(0, 0)  # Percepatan awal
        self.radius = mass * 3  # Ukuran bola proporsional dengan massa
        self.color = color

        # Gambar bola pertama kali di posisi awal
        x1, y1 = x - self.radius, y - self.radius
        x2, y2 = x + self.radius, y + self.radius
        self.body = canvas.create_oval(x1, y1, x2, y2, fill=color)

    def apply_force(self, force):
        """Menerapkan gaya ke benda (a = F/m)"""
        f = force.copy()         # Salin gaya
        f.div(self.mass)         # Hitung percepatan dari gaya: a = F/m
        self.acceleration.add(f) # Tambahkan percepatan ke benda

    def update(self):
        """Perbarui kecepatan dan posisi"""
        self.velocity.add(self.acceleration)  # v = v + a
        self.location.add(self.velocity)      # pos = pos + v
        self.acceleration = Vector(0, 0)      # Reset percepatan

        # Perbarui posisi bola menggunakan canvas.coords
        x, y, r = self.location.x, self.location.y, self.radius
        self.canvas.coords(self.body, x - r, y - r, x + r, y + r)

    def get_info(self):
        """Mengembalikan info perhitungan untuk ditampilkan"""
        return (
            f"Mass: {self.mass} kg\n"
            f"Gaya: (1.00, 0.00) N\n"
            f"Percepatan (a = F/m): {Vector(1,0).copy().div(self.mass) or Vector(1,0)}\n"
            f"Kecepatan: {self.velocity}\n"
            f"Posisi: {self.location}\n"
        )

# -----------------------------
# Class Simulation
# -----------------------------
class Simulation:
    def __init__(self, root):
        self.root = root
        self.width = 500
        self.height = 400

        # Canvas utama untuk menggambar bola
        self.canvas = tk.Canvas(root, width=self.width, height=self.height, bg="white")
        self.canvas.pack()

        # Label teks di bawah canvas untuk perhitungan fisika
        self.info_label = tk.Label(root, text="", font=("Courier", 10), justify="left", anchor="w")
        self.info_label.pack(padx=10, anchor="w")

        # Inisialisasi dua bola dengan massa berbeda
        self.mover1 = Mover(self.canvas, mass=5, x=100, y=200, color="blue")
        self.mover2 = Mover(self.canvas, mass=15, x=100, y=100, color="red")

        # Gaya konstan ke kanan (angin)
        self.wind = Vector(1, 0)

        # Jalankan simulasi
        self.update()

    def update(self):
        # Terapkan gaya angin ke masing-masing bola
        self.mover1.apply_force(self.wind)
        self.mover2.apply_force(self.wind)

        # Perbarui posisi dan kecepatan masing-masing bola
        self.mover1.update()
        self.mover2.update()

        # Buat teks gabungan untuk kedua benda
        info_text = "--- Bola Biru (massa kecil) ---\n"
        info_text += self.mover1.get_info()
        info_text += "\n--- Bola Merah (massa besar) ---\n"
        info_text += self.mover2.get_info()

        # Tampilkan teks di bawah canvas
        self.info_label.config(text=info_text)

        # Lanjutkan animasi tiap 30 ms
        self.root.after(30, self.update)

# -----------------------------
# Jalankan program utama
# -----------------------------
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Simulasi Massa dan Gaya (Newton II) dengan Tkinter")
    sim = Simulation(root)
    root.mainloop()



In [20]:
# SCRIPT 8 - Simulasi Hukum Kedua Newton (F = m * a) dengan Static Method
import tkinter as tk
from vector import Vector

# -----------------------------
# Class Mover untuk benda yang bergerak
# -----------------------------
class Mover:
    def __init__(self, canvas, mass, x, y, color):
        self.canvas = canvas
        self.mass = mass  # Massa benda (kg)
        self.location = Vector(x, y)  # Posisi awal
        self.velocity = Vector(0, 0)  # Kecepatan awal
        self.acceleration = Vector(0, 0)  # Percepatan awal
        self.radius = mass * 3  # Ukuran bola tergantung massa
        self.color = color

        # Gambar awal bola di canvas dan simpan ID-nya
        r = self.radius
        self.body = canvas.create_oval(x - r, y - r, x + r, y + r, fill=color)

    def apply_force(self, force):
        """Menerapkan gaya ke benda menggunakan static method div"""
        f = Vector.dived(force, self.mass)  # Hitung percepatan: a = F/m
        self.acceleration.add(f)  # Tambahkan percepatan

    def update(self):
        """Update posisi dan kecepatan benda"""
        self.velocity.add(self.acceleration)  # v = v + a
        self.location.add(self.velocity)      # pos = pos + v
        self.acceleration = Vector(0, 0)      # Reset percepatan

        # Perbarui posisi bola dengan canvas.coords (tidak menggambar ulang)
        x, y, r = self.location.x, self.location.y, self.radius
        self.canvas.coords(self.body, x - r, y - r, x + r, y + r)

    def get_info(self):
        """Mengembalikan info perhitungan untuk ditampilkan"""
        return (
            f"Mass: {self.mass} kg\n"
            f"Gaya: (1.00, 0.00) N\n"
            f"Percepatan (a = F/m): {Vector(1,0).copy().div(self.mass) or Vector(1,0)}\n"
            f"Kecepatan: {self.velocity}\n"
            f"Posisi: {self.location}\n"
        )

# -----------------------------
# Class Simulation untuk menjalankan animasi
# -----------------------------
class Simulation:
    def __init__(self, root):
        self.root = root
        self.width = 400
        self.height = 400

        # Canvas utama untuk menggambar bola
        self.canvas = tk.Canvas(root, width=self.width, height=self.height, bg="white")
        self.canvas.pack()

        # Label teks di bawah canvas untuk perhitungan fisika
        self.info_label = tk.Label(root, text="", font=("Courier", 10), justify="left", anchor="w")
        self.info_label.pack(padx=10, anchor="w")

        # Buat dua bola dengan massa berbeda
        self.mover1 = Mover(self.canvas, mass=5, x=100, y=200, color="blue")   # Bola biru, massa kecil
        self.mover2 = Mover(self.canvas, mass=15, x=100, y=300, color="red")   # Bola merah, massa besar

        # Gaya angin ke kanan sebesar 1 Newton
        self.wind = Vector(1, 0)

        # Mulai animasi
        self.update()

    def update(self):
        # Terapkan gaya ke kedua bola
        self.mover1.apply_force(self.wind)
        self.mover2.apply_force(self.wind)

        # Perbarui posisi kedua bola
        self.mover1.update()
        self.mover2.update()

        # Buat teks gabungan untuk kedua benda
        info_text = "--- Bola Biru (massa kecil) ---\n"
        info_text += self.mover1.get_info()
        info_text += "\n--- Bola Merah (massa besar) ---\n"
        info_text += self.mover2.get_info()

        # Tampilkan teks di bawah canvas
        self.info_label.config(text=info_text)
        
        # Jadwalkan frame berikutnya setelah 30 ms
        self.root.after(30, self.update)

# -----------------------------
# Jalankan program utama
# -----------------------------
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Simulasi Hukum Newton II (F = m * a) dengan Static Method")
    sim = Simulation(root)
    root.mainloop()



In [ ]:
# SCRIPT 9 - Simulasi gaya dengan variabel angin dan massa
import tkinter as tk
from vector import Vector

# === CLASS MOVER UNTUK BENDA BERGERAK ===
class Mover:
    def __init__(self, mass, x, y):
        self.mass = mass                              # Massa benda (kg)
        self.position = Vector(x, y)                  # Posisi awal (x, y)
        self.velocity = Vector(0, 0)                  # Kecepatan awal (diam)
        self.acceleration = Vector(0, 0)              # Percepatan awal nol
        self.radius = mass * 10                       # Ukuran bola sebanding dengan massa

    def apply_force(self, force):
        """Menerapkan gaya ke benda: a = F / m"""
        a = Vector.div(force, self.mass)              # Hitung percepatan dari gaya
        self.acceleration.add(a)                      # Tambahkan ke total percepatan

    def update(self):
        """Update posisi berdasarkan percepatan dan kecepatan"""
        self.velocity.add(self.acceleration)          # v = v + a
        self.position.add(self.velocity)              # s = s + v
        self.acceleration = Vector(0, 0)              # Reset percepatan

    def check_edges(self, height):
        """Pantulkan jika menyentuh dasar canvas"""
        if self.position.y > height - self.radius:
            self.position.y = height - self.radius
            self.velocity.y *= -1                     # Pantulan: arah kecepatan vertikal dibalik

# === CLASS UTAMA UNTUK SIMULASI ===
class Simulation:
    def __init__(self, root):
        self.root = root
        self.width = 600
        self.height = 400

        # Membuat canvas
        self.canvas = tk.Canvas(root, width=self.width, height=self.height, bg="white")
        self.canvas.pack()

        # Membuat objek bola
        self.mover = Mover(mass=2, x=100, y=50)

        # Gaya tetap
        self.gravity = Vector(0, 0.3)      # Gravitasi ke bawah
        self.wind = Vector(0.1, 0)         # Angin ke kanan

        self.mouse_pressed = False         # Status mouse ditekan

        # Event mouse
        self.root.bind("<ButtonPress-1>", self.on_mouse_down)
        self.root.bind("<ButtonRelease-1>", self.on_mouse_up)

        # Gambar bola pertama kali menggunakan create_oval
        r = self.mover.radius
        x, y = self.mover.position.x, self.mover.position.y
        self.ball = self.canvas.create_oval(x - r, y - r, x + r, y + r, fill="skyblue")

        # Label informasi
        self.info_label = tk.Label(root, font=("Courier", 10), justify="left", bg="white", anchor="w")
        self.info_label.pack(fill="both")

        # Mulai update loop
        self.update()

    def on_mouse_down(self, event):
        """Deteksi ketika mouse diklik"""
        self.mouse_pressed = True

    def on_mouse_up(self, event):
        """Deteksi ketika mouse dilepas"""
        self.mouse_pressed = False

    def update(self):
        # === 1. Hitung gaya total ===
        total_force = self.gravity.copy()         # Awalnya hanya gravitasi
        if self.mouse_pressed:
            total_force.add(self.wind)            # Tambahkan angin jika mouse ditekan

        # === 2. Terapkan gaya ke benda ===
        self.mover.apply_force(total_force)

        # === 3. Update posisi, kecepatan, dan cek pantulan ===
        self.mover.update()
        self.mover.check_edges(self.height)

        # === 4. Update posisi bola di canvas menggunakan canvas.coords ===
        r = self.mover.radius
        x = self.mover.position.x
        y = self.mover.position.y
        self.canvas.coords(self.ball, x - r, y - r, x + r, y + r)  # Menggeser bola ke posisi baru

        # === 5. Hitung percepatan (a = F / m) untuk ditampilkan ===
        fx = total_force.x
        fy = total_force.y
        mass = self.mover.mass
        ax = fx / mass
        ay = fy / mass

        # === 6. Tampilkan informasi fisika di label ===
        info = f"""
Rumus: a = F / m

Gaya total:
  Fx = {fx:.2f}, Fy = {fy:.2f}
Percepatan:
  ax = Fx/m = {fx:.2f}/{mass} = {ax:.2f}
  ay = Fy/m = {fy:.2f}/{mass} = {ay:.2f}
Kecepatan:
  vx = {self.mover.velocity.x:.2f}
  vy = {self.mover.velocity.y:.2f}
Posisi:
  x = {x:.2f}
  y = {y:.2f}

Klik kiri untuk menambah gaya angin ke kanan.
"""
        self.info_label.config(text=info)

        # === 7. Ulangi update setelah 30ms ===
        self.root.after(30, self.update)

# === JALANKAN PROGRAM ===
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Simulasi Gaya dan Percepatan (F = m × a)")
    Simulation(root)
    root.mainloop()

In [27]:
# SCRIPT 10 - Simulasi gaya dengan variabel angin dan massa utk 4 bola
import tkinter as tk
from vector import Vector

# === CLASS MOVER UNTUK OBJEK BOLA BERGERAK ===
class Mover:
    def __init__(self, mass, x, y, color="skyblue"):
        self.mass = mass                            # Massa bola
        self.position = Vector(x, y)                # Posisi awal bola
        self.velocity = Vector(0, 0)                # Kecepatan awal (vx = 0, vy = 0)
        self.acceleration = Vector(0, 0)            # Percepatan awal
        self.radius = mass * 10                     # Radius bola bergantung pada massa
        self.color = color                          # Warna bola
        self.id = None                              # ID bola di canvas (diset nanti)

    def apply_force(self, force):
        # Hitung percepatan: a = F / m dan tambahkan ke total percepatan
        a = Vector.div(force, self.mass)
        self.acceleration.add(a)

    def update(self):
        # Update kecepatan dan posisi, lalu reset percepatan
        self.velocity.add(self.acceleration)
        self.position.add(self.velocity)
        self.acceleration = Vector(0, 0)

    def check_edges(self, height):
        # Jika bola jatuh melewati bawah canvas, pantulkan ke atas
        if self.position.y > height - self.radius:
            self.position.y = height - self.radius
            self.velocity.y *= -1

# === CLASS SIMULASI UTAMA ===
class Simulation:
    def __init__(self, root):
        self.root = root
        self.width = 800
        self.height = 500
        self.canvas = tk.Canvas(root, width=self.width, height=self.height, bg="white")
        self.canvas.pack()

        # Buat 4 bola dengan massa berbeda
        self.movers = [
            Mover(1, 100, 50, "red"),
            Mover(2, 250, 50, "green"),
            Mover(3, 400, 50, "blue"),
            Mover(4, 550, 50, "orange"),
        ]

        # Gaya konstan
        self.gravity = Vector(0, 0.3)  # Gaya gravitasi ke bawah
        self.wind = Vector(0.1, 0)     # Gaya angin ke kanan
        self.mouse_pressed = False     # Status mouse ditekan

        # Buat bola-bola di canvas dan simpan ID-nya
        for mover in self.movers:
            r = mover.radius
            x, y = mover.position.x, mover.position.y
            mover.id = self.canvas.create_oval(x - r, y - r, x + r, y + r, fill=mover.color)

        # Event mouse untuk angin
        self.root.bind("<ButtonPress-1>", self.on_mouse_down)
        self.root.bind("<ButtonRelease-1>", self.on_mouse_up)

        self.update()  # Jalankan animasi

    def on_mouse_down(self, event):
        self.mouse_pressed = True  # Ketika mouse ditekan, angin aktif

    def on_mouse_up(self, event):
        self.mouse_pressed = False  # Saat mouse dilepas, angin berhenti

    def update(self):
        self.canvas.delete("text")  # Hapus teks sebelumnya (bukan bola)

        text_y = 20  # Posisi awal teks keterangan

        for mover in self.movers:
            # === 1. Hitung gaya total ===
            total_force = self.gravity.copy()
            if self.mouse_pressed:
                total_force.add(self.wind)

            # === 2. Terapkan gaya, update posisi dan periksa tepi ===
            mover.apply_force(total_force)
            mover.update()
            mover.check_edges(self.height)

            # === 3. Update posisi bola menggunakan canvas.coords ===
            x, y, r = mover.position.x, mover.position.y, mover.radius
            self.canvas.coords(mover.id, x - r, y - r, x + r, y + r)

            # === 4. Hitung dan tampilkan informasi gaya, a, v, posisi ===
            fx, fy = total_force.x, total_force.y
            mass = mover.mass
            ax, ay = fx / mass, fy / mass

            info = (
                f"Bola mass={mass} | "
                f"F: ({fx:.2f}, {fy:.2f}) | "
                f"a: ({ax:.2f}, {ay:.2f}) | "
                f"v: ({mover.velocity.x:.2f}, {mover.velocity.y:.2f}) | "
                f"pos: ({x:.2f}, {y:.2f})"
            )
            # Gambar teks info di bawah canvas
            self.canvas.create_text(400, text_y, text=info, font=("Courier", 10), anchor="n", tags="text")
            text_y += 18

        # === 5. Teks petunjuk pengguna ===
        self.canvas.create_text(
            400, self.height - 20,
            text="Klik kiri mouse untuk menambah gaya angin ke kanan (wind)",
            font=("Arial", 10), fill="black", tags="text"
        )

        # === 6. Looping animasi setiap 30 ms ===
        self.root.after(30, self.update)

# === JALANKAN PROGRAM ===
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Simulasi Gaya dan Gerakan - 4 Bola (canvas.coords)")
    Simulation(root)
    root.mainloop()

In [35]:
# SCRIPT 11 - Simulasi dengan invisible edge push (revisi dengan canvas.coords)
import tkinter as tk
from vector import Vector
# ======================
# Kelas Bola (objek fisik)
class Mover:
    def __init__(self, mass, x, y, color="skyblue"):
        self.mass = mass
        self.position = Vector(x, y)             # Posisi awal
        self.velocity = Vector(0, 0)             # Kecepatan awal
        self.acceleration = Vector(0, 0)         # Percepatan awal
        self.radius = mass * 10                  # Radius berdasarkan massa
        self.color = color                       # Warna bola

    def apply_force(self, force):
        # Menghitung percepatan: a = F / m
        a = Vector.dived(force, self.mass)
        self.acceleration.add(a)

    def apply_edge_push(self, width, height, margin=50, strength=0.05):
        # Menambahkan gaya dorong jika bola terlalu dekat tepi
        if self.position.x < margin:
            push = Vector((margin - self.position.x) * strength, 0)
            self.apply_force(push)
        if self.position.x > width - margin:
            push = Vector((width - margin - self.position.x) * strength, 0)
            self.apply_force(push)
        if self.position.y < margin:
            push = Vector(0, (margin - self.position.y) * strength)
            self.apply_force(push)
        if self.position.y > height - margin:
            push = Vector(0, (height - margin - self.position.y) * strength)
            self.apply_force(push)

    def update(self):
        # Update kecepatan dan posisi berdasarkan percepatan
        self.velocity.add(self.acceleration)
        self.position.add(self.velocity)
        self.acceleration = Vector(0, 0)  # Reset percepatan setiap frame

# ======================
# Kelas Simulasi
class Simulation:
    def __init__(self, root):
        self.width = 600
        self.height = 400
        self.canvas = tk.Canvas(root, width=self.width, height=self.height, bg="white")
        self.canvas.pack()

        # Membuat bola
        self.mover = Mover(2, 100, 300, color="orange")

        # Gaya gravitasi dan angin
        self.gravity = Vector(0, 0.2)
        self.wind = Vector(0.05, 0)

        # ID untuk objek canvas
        self.ball_id = None
        self.text_id = None
        self.instruction_id = None

        self.draw()  # Mulai loop animasi

    def draw(self):
        # Reset gaya total setiap frame
        total_force = Vector()

        # Terapkan gaya gravitasi
        self.mover.apply_force(self.gravity)
        total_force.add(self.gravity)

        # Terapkan gaya angin
        self.mover.apply_force(self.wind)
        total_force.add(self.wind)

        # Terapkan dorongan jika dekat tepi
        self.mover.apply_edge_push(self.width, self.height)

        # Update posisi dan kecepatan
        self.mover.update()

        # Ambil posisi dan radius bola
        x, y = self.mover.position.x, self.mover.position.y
        r = self.mover.radius

        # Gambar bola atau update posisi menggunakan canvas.coords
        if self.ball_id is None:
            self.ball_id = self.canvas.create_oval(x - r, y - r, x + r, y + r, fill=self.mover.color)
        else:
            self.canvas.coords(self.ball_id, x - r, y - r, x + r, y + r)  # Update posisi bola

        # Hitung data fisika
        fx, fy = total_force.x, total_force.y
        ax = fx / self.mover.mass
        ay = fy / self.mover.mass

        info_text = (
            f"Rumus: a = F / m\n"
            f"Ftotal = ({fx:.2f}, {fy:.2f})\n"
            f"Percepatan = ({ax:.2f}, {ay:.2f})\n"
            f"Kecepatan = ({self.mover.velocity.x:.2f}, {self.mover.velocity.y:.2f})\n"
            f"Posisi = ({x:.2f}, {y:.2f})"
        )

        # Gambar atau update teks info fisika
        if self.text_id is None:
            self.text_id = self.canvas.create_text(10, 10, anchor="nw", font=("Courier", 12), text=info_text)
        else:
            self.canvas.itemconfig(self.text_id, text=info_text)

        # Gambar atau update instruksi (gunakan coords untuk jaga posisinya tetap)
        if self.instruction_id is None:
            self.instruction_id = self.canvas.create_text(
                self.width / 2, self.height - 20,
                text="Dorongan tepi akan muncul jika bola terlalu dekat tepi",
                font=("Arial", 10), fill="gray"
            )
        else:
            self.canvas.coords(self.instruction_id, self.width / 2, self.height - 20)

        # Loop animasi setiap 30 ms
        self.canvas.after(30, self.draw)

# ======================
# Jalankan program
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Simulasi Gaya Sederhana - 1 Bola dengan Edge Push")
    Simulation(root)
    root.mainloop()



In [36]:
# SCRIPT 12 - Simulasi Gravitasi dengan Tkinter
import tkinter as tk
from vector import Vector

# ========================
# Class Mover: mewakili bola atau benda yang jatuh
# ========================
class Mover:
    def __init__(self, mass, x, y):
        self.mass = mass
        self.radius = mass * 5  # Ukuran bola berdasarkan massa
        self.location = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)

        # Simpan ID untuk menggambar objek canvas
        self.oval_id = None      # ID untuk bola
        self.label_id = None     # ID untuk teks massa di tengah bola
        self.info_id = None      # ID untuk info fisika di bawah canvas

    # Fungsi untuk menambahkan gaya pada benda (F/m = a)
    def apply_force(self, force):
        f = Vector.dived(force, self.mass)
        self.acceleration.add(f)

    # Perbarui kecepatan dan posisi benda
    def update(self):
        self.velocity.add(self.acceleration)
        self.location.add(self.velocity)
        self.acceleration = Vector(0, 0)  # Reset percepatan setelah update

    # Gambar atau update posisi bola dan label massa
    def display(self, canvas):
        x, y = self.location.x, self.location.y
        r = self.radius

        # Gambar atau update posisi bola
        if self.oval_id is None:
            self.oval_id = canvas.create_oval(x - r, y - r, x + r, y + r, fill="skyblue")
        else:
            canvas.coords(self.oval_id, x - r, y - r, x + r, y + r)  # Perbarui posisi bola

        # Gambar atau update teks massa
        if self.label_id is None:
            self.label_id = canvas.create_text(x, y, text=f"{self.mass}kg", font=("Arial", 10), fill="black")
        else:
            canvas.coords(self.label_id, x, y)  # Perbarui posisi teks

    # Gambar atau update info fisika di bawah layar
    def display_info(self, canvas, offset_y):
        fg = self.mass * 0.1  # Gaya gravitasi (g = 0.1)
        a = fg / self.mass    # Percepatan = F/m
        v = self.velocity.y   # Kecepatan vertikal

        info_text = f"Massa = {self.mass} kg, Fg = {fg:.2f} N, a = {a:.2f} m/s², v = {v:.2f} m/s"

        if self.info_id is None:
            self.info_id = canvas.create_text(
                100, offset_y,
                anchor="w", font=("Arial", 10),
                fill="black", text=info_text
            )
        else:
            canvas.itemconfig(self.info_id, text=info_text)  # Update teks
            canvas.coords(self.info_id, 100, offset_y)       # Update posisi teks

# ========================
# Class GravitySimulation: untuk menjalankan simulasi
# ========================
class GravitySimulation:
    def __init__(self, root):
        self.root = root
        self.WIDTH = 600
        self.HEIGHT = 400

        self.canvas = tk.Canvas(root, width=self.WIDTH, height=self.HEIGHT, bg="white")
        self.canvas.pack()

        # Buat 2 bola dengan massa berbeda
        self.movers = [
            Mover(mass=10, x=150, y=50),
            Mover(mass=2, x=350, y=50)
        ]

        self.gravity = Vector(0, 0.1)  # Gaya gravitasi ke bawah

        self.update()  # Mulai simulasi

    def update(self):
        for i, mover in enumerate(self.movers):
            # Hitung gaya gravitasi dan terapkan ke masing-masing benda
            gravity_force = Vector.multed(self.gravity, mover.mass)
            mover.apply_force(gravity_force)

            mover.update()  # Update posisi berdasarkan gaya
            mover.display(self.canvas)  # Gambar atau update bola
            mover.display_info(self.canvas, offset_y=300 + i * 20)  # Info di bawah

        self.root.after(30, self.update)  # Loop update setiap 30 ms

# ========================
# Jalankan program
# ========================
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Simulasi Gravitasi Berdasarkan Massa")
    app = GravitySimulation(root)
    root.mainloop()

In [37]:
# SCRIPT 13 - Simulasi Gaya Gesek dengan Tkinter
import tkinter as tk
from vector import Vector

# ======== Class Mover: Benda yang Diberi Gaya ========
class Mover:
    def __init__(self, canvas, x, y, mass):
        self.canvas = canvas               # Simpan referensi canvas
        self.mass = mass                   # Massa benda
        self.radius = mass * 8             # Radius tergantung massa
        self.position = Vector(x, y)       # Posisi awal
        self.velocity = Vector(0, 0)       # Kecepatan awal
        self.acceleration = Vector(0, 0)   # Percepatan awal

        # Gambar bola sekali, dan simpan id objeknya
        self.id = self.canvas.create_oval(
            x - self.radius, y - self.radius,
            x + self.radius, y + self.radius,
            fill="skyblue", outline="black", width=2
        )

    def apply_force(self, force):
        # Rumus Newton: a = F / m
        f = Vector.dived(force, self.mass)
        self.acceleration.add(f)

    def update(self):
        # Tambah percepatan ke kecepatan, lalu ke posisi
        self.velocity.add(self.acceleration)
        self.position.add(self.velocity)
        self.acceleration = Vector(0, 0)  # Reset percepatan tiap frame

        # Perbarui posisi gambar bola di canvas
        x, y, r = self.position.x, self.position.y, self.radius
        self.canvas.coords(self.id, x - r, y - r, x + r, y + r)

    def contact_edge(self, height):
        # Deteksi jika menyentuh bawah
        return self.position.y + self.radius >= height

    def bounce_edges(self, width, height):
        bounce = -0.9
        if self.position.x > width - self.radius:
            self.position.x = width - self.radius
            self.velocity.x *= bounce
        elif self.position.x < self.radius:
            self.position.x = self.radius
            self.velocity.x *= bounce

        if self.position.y > height - self.radius:
            self.position.y = height - self.radius
            self.velocity.y *= bounce

# ======== Setup Simulasi Tkinter ========
WIDTH, HEIGHT = 640, 400
root = tk.Tk()
root.title("Simulasi Gaya Gesek - The Nature of Code")
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# Label teks untuk info fisika
text = tk.Label(root, text="", font=("Courier", 10), justify="left", anchor="nw")
text.pack()

# Buat objek Mover (benda) dengan massa 5
mover = Mover(canvas, WIDTH // 2, 30, mass=5)

# Flag untuk mendeteksi mouse ditekan
mouse_pressed = [False]

# ======== Fungsi Simulasi Frame-by-Frame ========
def draw():
    # Tidak perlu hapus bola dari canvas karena pakai canvas.coords

    # 1. Gaya gravitasi konstan ke bawah
    gravity = Vector(0, 1)
    mover.apply_force(gravity)
    result = f"Gaya:\n  Gravitasi : {gravity}"

    # 2. Gaya angin ke kanan jika mouse ditekan
    if mouse_pressed[0]:
        wind = Vector(0.2, 0)
        mover.apply_force(wind)
        result += f"\n  Angin     : {wind}"

    # 3. Gaya gesek saat menyentuh lantai >> hilangkan jika tidak ingin ada friksi
    if mover.contact_edge(HEIGHT):
        c = 0.5
        friction = mover.velocity.copy()  # Ambil arah kecepatan
        friction.mult(-1)                 # Balik arah
        friction.normalize()              # Jadikan unit vector
        friction.mult(c)                  # Kalikan dengan koefisien
        mover.apply_force(friction)
        result += f"\n  Gesekan   : {friction}"

    # 4. Update posisi, cek pantulan, dan tampilkan
    mover.update()
    mover.bounce_edges(WIDTH, HEIGHT)

    # 5. Tampilkan info numerik
    result += f"\n\nPercepatan : {mover.acceleration}"
    result += f"\nKecepatan  : {mover.velocity}"
    result += f"\nPosisi     : {mover.position}"
    text.config(text=result)

    # Jalankan kembali setelah 30 ms
    root.after(30, draw)

# ======== Fungsi Interaksi Mouse ========
def on_press(event): mouse_pressed[0] = True
def on_release(event): mouse_pressed[0] = False
canvas.bind("<ButtonPress-1>", on_press)
canvas.bind("<ButtonRelease-1>", on_release)

# ======== Jalankan Simulasi ========
draw()
root.mainloop()


In [ ]:
# SCRIPT 14 - Simulasi Bola jatuh di Cairan
import tkinter as tk
import random
from vector import Vector
# ------------ Kelas Cairan (Liquid) ------------
class Liquid:
    def __init__(self, x, y, w, h, c):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.c = c  # Koefisien drag (resistance)

    # Cek apakah mover berada di dalam cairan
    def contains(self, mover):
        pos = mover.position
        return self.x < pos.x < self.x + self.w and self.y < pos.y < self.y + self.h

    # Hitung gaya hambat fluida (drag)
    def calculate_drag(self, mover):
        speed = mover.velocity.mag()
        drag_magnitude = self.c * speed ** 2

        # Arah drag berlawanan dengan arah kecepatan
        drag = mover.velocity.copy()
        drag.mult(-1)
        drag.set_mag(drag_magnitude)
        return drag

    # Gambar cairan di canvas
    def show(self, canvas):
        canvas.create_rectangle(self.x, self.y, self.x + self.w, self.y + self.h, fill="#ccf")

# ------------ Kelas Benda Bergerak (Mover) ------------
class Mover:
    def __init__(self, x, y, mass):
        self.mass = mass
        self.radius = mass * 8
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)

    # Terapkan gaya ke benda (F = m * a → a = F / m)
    def apply_force(self, force):
        f = force.copy()
        f.div(self.mass)
        self.acceleration.add(f)

    # Update posisi berdasarkan kecepatan dan percepatan
    def update(self):
        self.velocity.add(self.acceleration)
        self.position.add(self.velocity)
        self.acceleration = Vector(0, 0)  # Reset percepatan setelah digunakan

    # Gambar benda di canvas
    def show(self, canvas):
        x = self.position.x
        y = self.position.y
        r = self.radius
        canvas.create_oval(x - r, y - r, x + r, y + r, fill="gray", outline="black")

    # Cek apakah menyentuh lantai, jika ya, pantul
    def check_edges(self, height):
        if self.position.y > height - self.radius:
            self.velocity.y *= -0.9
            self.position.y = height - self.radius

# ------------ Program Simulasi ------------
class Simulation:
    def __init__(self, root):
        self.root = root
        self.width = 640
        self.height = 360
        self.canvas = tk.Canvas(root, width=self.width, height=self.height)
        self.canvas.pack()
        self.movers = []
        self.liquid = Liquid(0, self.height // 2, self.width, self.height // 2, 0.1)
        self.reset()

        self.root.bind("<Button-1>", self.mouse_pressed)
        self.draw()

    # #Buat ulang objek mover secara acak
    # def reset(self):
    #     self.movers = []
    #     for i in range(9):
    #         x = 40 + i * 60
    #         m = random.uniform(0.5, 3)
    #         self.movers.append(Mover(x, 0, m))

    # Exercise 2.5 – Variasi Ketinggian Jatuh
    def reset(self):
        self.movers = []
        # Tiga bola dengan ketinggian berbeda
        heights = [50, 100, 150]  # Tiga ketinggian berbeda
        for y in heights:
            x_pos = random.randint(100, 700)
            m = random.uniform(1, 2.5)  # Massa acak sedang
            self.movers.append(Mover(x_pos, y, m))

    # Jika mouse diklik, ulang simulasi
    def mouse_pressed(self, event):
        self.reset()

    # Fungsi utama untuk menggambar setiap frame
    def draw(self):
        self.canvas.delete("all")  # Hapus semua gambar lama
        self.liquid.show(self.canvas)  # Gambar cairan

        for mover in self.movers:
            # Jika mover masuk cairan, terapkan gaya drag
            if self.liquid.contains(mover):
                drag = self.liquid.calculate_drag(mover)
                mover.apply_force(drag)

            # Gaya gravitasi ke bawah (F = m * g)
            gravity = Vector(0, 0.1 * mover.mass)
            mover.apply_force(gravity)

            mover.update()
            mover.check_edges(self.height)
            mover.show(self.canvas)

        # Jalankan ulang fungsi draw tiap 16 ms (sekitar 60 fps)
        self.root.after(16, self.draw)

# ------------ Jalankan Program ------------
root = tk.Tk()
root.title("Simulasi Fluid Resistance (Sederhana)")
app = Simulation(root)
root.mainloop()


In [38]:
# SCRIPT 15 - Simulasi Gaya Drag pada Kotak di Cairan
import tkinter as tk
from vector import Vector

# ======== Box Class ========
class Box:
    def __init__(self, x, y, w, h, mass):
        self.width = w
        self.height = h
        self.mass = mass
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
    def apply_force(self, force):
        f = Vector.dived(force, self.mass)
        self.acceleration.add(f)
    def update(self):
        self.velocity.add(self.acceleration)
        self.position.add(self.velocity)
        self.acceleration = Vector(0, 0)
    def show(self, canvas):
        canvas.coords(self.id,
            self.position.x - self.width / 2,
            self.position.y - self.height / 2,
            self.position.x + self.width / 2,
            self.position.y + self.height / 2)
    def set_id(self, canvas, color="orange"):
        self.id = canvas.create_rectangle(
            self.position.x - self.width / 2,
            self.position.y - self.height / 2,
            self.position.x + self.width / 2,
            self.position.y + self.height / 2,
            fill=color, outline="black", width=2
        )

# ======== Setup Tkinter ========
WIDTH, HEIGHT = 640, 480
root = tk.Tk()
root.title("Box Drag Force with Surface Area")
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

fluid_top = HEIGHT // 2
canvas.create_rectangle(0, fluid_top, WIDTH, HEIGHT, fill="lightblue", outline="")

# Dua kotak dengan area bawah berbeda
boxes = [
    Box(200, 100, w=40, h=60, mass=2),
    Box(400, 100, w=100, h=60, mass=2),
]

for box in boxes:
    box.set_id(canvas)

# ======== Loop Simulasi ========
def draw():
    for box in boxes:
        gravity = Vector(0, 0.5)
        box.apply_force(gravity)

        # Cek apakah kotak masuk ke area air
        if box.position.y + box.height/2 >= fluid_top:
            c = 0.0005  # koefisien drag lebih kecil agar tidak terlalu kuat
            speed = box.velocity.mag()
            if speed > 0:
                drag_dir = box.velocity.copy()
                drag_dir.normalize()
                drag_dir.mult(-1)  # arah drag berlawanan

                A = box.width  # permukaan menyentuh air
                drag_magnitude = c * A * speed * speed
                drag = drag_dir
                drag.mult(drag_magnitude)

                box.apply_force(drag)

        box.update()
        box.show(canvas)

    root.after(30, draw)

draw()
root.mainloop()


In [39]:
# SCRIPT 16 - Simulasi Gaya Lift dan Drag pada Sayap Pesawat
import tkinter as tk
from vector import Vector

# ======= Sayap Pesawat =======
class Wing:
    def __init__(self, x, y):
        self.position = Vector(x, y)
        self.velocity = Vector(5, 0)
        self.acceleration = Vector(7, 0)
        self.width = 60
        self.height = 20
        self.mass = 1
    def apply_force(self, force):
        f = Vector(force.x / self.mass, force.y / self.mass)
        self.acceleration.add(f)
    def update(self):
        self.velocity.add(self.acceleration)
        #self.velocity.limit(30)
        self.position.add(self.velocity)
        self.acceleration = Vector(0, 0)
    def show(self, canvas):
        canvas.coords(self.id,
            self.position.x - self.width/2,
            self.position.y - self.height/2,
            self.position.x + self.width/2,
            self.position.y + self.height/2)
    def create(self, canvas):
        self.id = canvas.create_rectangle(
            self.position.x - self.width/2,
            self.position.y - self.height/2,
            self.position.x + self.width/2,
            self.position.y + self.height/2,
            fill="red"
        )

# ======= Setup Tkinter =======
WIDTH, HEIGHT = 800, 600
root = tk.Tk()
root.title("Simulasi Gaya Lift dan Drag")
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# Buat sayap
wing = Wing(100, HEIGHT/2)
wing.create(canvas)

# Tombol keyboard
key_pressed = {"Up": False}
def key_down(event):
    if event.keysym == "Up":
        key_pressed["Up"] = True
def key_up(event):
    if event.keysym == "Up":
        key_pressed["Up"] = False
root.bind("<KeyPress>", key_down)
root.bind("<KeyRelease>", key_up)

# ======= Loop utama =======
def draw():
    canvas.delete("info")

    # Gaya gravitasi
    gravity = Vector(0, 0.1)
    wing.apply_force(gravity)

    # Gaya drag (melawan gerak)
    drag = Vector(-wing.velocity.x * 0.02, -wing.velocity.y * 0.02)
    wing.apply_force(drag)

    # Gaya lift saat tombol ditekan
    if key_pressed["Up"]:
        lift = Vector(0, -0.6)
        wing.apply_force(lift)
    else:
        lift = Vector(0, 0)

    # Update gerakan & gambar
    wing.update()
    wing.show(canvas)

    # Info di layar
    canvas.create_text(10, 10, anchor="nw", text=f"Kecepatan: ({wing.velocity.x:.2f}, {wing.velocity.y:.2f})", tag="info")
    canvas.create_text(10, 30, anchor="nw", text=f"Lift Aktif: {'Ya' if key_pressed['Up'] else 'Tidak'}", tag="info")

    root.after(30, draw)

draw()
root.mainloop()

In [ ]:
# SCRIPT 17 - Simulasi Gaya Tarik antara Dua Benda
import tkinter as tk
from vector import Vector

# === CLASS MOVER (BENDA YANG DITARIK) ===
class Mover:
    def __init__(self, x, y, mass, canvas):
        self.location = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.mass = mass
        self.canvas = canvas
        self.radius = mass * 2

        self.id = canvas.create_oval(x - self.radius, y - self.radius,
                                     x + self.radius, y + self.radius,
                                     fill='skyblue')

    def apply_force(self, force):
        # F = m * a → a = F / m
        a = force.dived(self.mass)  # vektor percepatan
        self.acceleration = self.acceleration.added(a)

    def update(self):
        self.velocity = self.velocity.added(self.acceleration)
        self.location = self.location.added(self.velocity)
        self.acceleration = Vector(0, 0)

    def display(self):
        x, y, r = self.location.x, self.location.y, self.radius
        self.canvas.coords(self.id, x - r, y - r, x + r, y + r)

# === CLASS PENARIK (PUSAT GAYA) ===
class Attractor:
    def __init__(self, x, y, mass, canvas):
        self.location = Vector(x, y)
        self.mass = mass
        self.canvas = canvas
        self.radius = mass * 2
        self.G = 1  # konstanta gravitasi

        self.id = canvas.create_oval(x - self.radius, y - self.radius,
                                     x + self.radius, y + self.radius,
                                     fill='orange')

    def attract(self, m):
        # Gaya arah = posisi attractor - posisi mover
        force = self.location.subbed(m.location)
        distance = force.mag()
        distance = max(5, min(distance, 25))  # batasi jarak

        # Normalisasi arah dan hitung kekuatan gaya
        direction = force.normalized()
        strength = (self.G * self.mass * m.mass) / (distance ** 2)
        force_vector = direction.multed(strength)
        return force_vector

    def display(self):
        # Tidak perlu update posisi
        pass

# === SETUP TKINTER ===
WIDTH, HEIGHT = 600, 400
root = tk.Tk()
root.title("Simulasi Gaya Tarik Gravitasi")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# Label teks informasi
info_text = canvas.create_text(10, 10, anchor="nw", text="", font=("Courier", 10), fill="black")

# Objek utama
attractor = Attractor(WIDTH / 2, HEIGHT / 2, 20, canvas)
mover = Mover(100, 100, 4, canvas)

# === LOOP UTAMA ===
def draw():
    # Hitung gaya tarik
    force = attractor.attract(mover)
    mover.apply_force(force)
    mover.update()
    mover.display()

    # Update teks info
    info = (
        f"Gaya (F):        {force}\n"
        f"Percepatan (a):  {mover.acceleration}\n"
        f"Kecepatan (v):   {mover.velocity}\n"
        f"Posisi (s):      {mover.location}"
    )
    canvas.itemconfig(info_text, text=info)

    root.after(33, draw)

draw()
root.mainloop()


In [ ]:
# SCRIPT 18 - Simulasi Gravitasi Multi Mover
import tkinter as tk
from vector import Vector

# ======== CLASS MOVER (BENDA BERGERAK) ========
class Mover:
    def __init__(self, x, y, mass, canvas):
        self.location = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.mass = mass
        self.r = self.mass * 2  # Ukuran lingkaran
        self.canvas = canvas
        self.oval = canvas.create_oval(x - self.r, y - self.r,
                                       x + self.r, y + self.r,
                                       fill='skyblue')
        self.text = canvas.create_text(0, 0, anchor="nw", font=("Arial", 10), fill="black")

    def apply_force(self, force):
        # F = m * a => a = F / m
        f = force.dived(self.mass)  # tidak ubah force asli
        self.acceleration = self.acceleration.added(f)  # akumulasi percepatan

    def update(self):
        self.velocity = self.velocity.added(self.acceleration)
        self.location = self.location.added(self.velocity)
        self.acceleration = Vector(0, 0)

    def display(self):
        x, y = self.location.x, self.location.y
        r = self.r
        self.canvas.coords(self.oval, x - r, y - r, x + r, y + r)

        # Tampilkan informasi perhitungan di atas benda
        info = f"Pos:({x:.1f},{y:.1f})\nVel:({self.velocity.x:.1f},{self.velocity.y:.1f})"
        self.canvas.coords(self.text, x + r + 5, y - r)
        self.canvas.itemconfig(self.text, text=info)

# ======== CLASS ATTRACTOR (SUMBER GAYA TARIK) ========
class Attractor:
    def __init__(self, x, y, mass, canvas):
        self.location = Vector(x, y)
        self.mass = mass
        self.G = 1
        self.r = self.mass * 2
        self.canvas = canvas
        self.oval = canvas.create_oval(x - self.r, y - self.r,
                                       x + self.r, y + self.r,
                                       fill='orange')

    def attract(self, m):
        # Arah gaya: dari benda ke pusat
        force = self.location.subbed(m.location)
        distance = force.mag()
        distance = max(5, min(distance, 25))  # Clamp jarak
        direction = force.normalized()  # arah vektor
        strength = (self.G * self.mass * m.mass) / (distance ** 2)
        force = direction.multed(strength)
        return force

    def display(self):
        x, y = self.location.x, self.location.y
        r = self.r
        self.canvas.coords(self.oval, x - r, y - r, x + r, y + r)

# ======== SETUP TKINTER ========
WIDTH, HEIGHT = 600, 400
root = tk.Tk()
root.title("Simulasi Gravitasi - Multi Mover")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# Buat attractor dan daftar mover
attractor = Attractor(WIDTH / 2, HEIGHT / 2, 20, canvas)

movers = [
    Mover(100, 100, 4, canvas),
    Mover(200, 50, 6, canvas),
    Mover(150, 250, 3, canvas)
]

# Loop utama animasi
def draw():
    for mover in movers:
        force = attractor.attract(mover)
        mover.apply_force(force)
        mover.update()
        mover.display()

    attractor.display()
    root.after(33, draw)

draw()
root.mainloop()



In [ ]:
# SCRIPT 19 - Simulasi Gaya Tarik Buatan Sendiri
import tkinter as tk
import random
from vector import Vector

# ---- Objek yang bergerak (Mover) ----
class Mover:
    def __init__(self, mass, x, y, canvas):
        self.mass = mass
        self.location = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.trail = []

        self.r = self.mass * 3
        self.canvas = canvas
        self.oval = canvas.create_oval(x - self.r, y - self.r,
                                       x + self.r, y + self.r,
                                       fill="blue", outline="")

    def apply_force(self, force):
        f = force.multed(1 / self.mass)
        self.acceleration.add(f)

    def update(self):
        self.velocity.add(self.acceleration)
        self.location.add(self.velocity)
        self.acceleration = Vector(0, 0)

        self.trail.append((self.location.x, self.location.y))
        if len(self.trail) > 40:
            self.trail.pop(0)

    def display(self):
        # Update posisi oval dengan .coords()
        x, y, r = self.location.x, self.location.y, self.r
        self.canvas.coords(self.oval, x - r, y - r, x + r, y + r)

        # Gambar jejak (garis-garis)
        # for i in range(len(self.trail) - 1):
        #     x1, y1 = self.trail[i]
        #     x2, y2 = self.trail[i + 1]
        #     self.canvas.create_line(x1, y1, x2, y2, fill="skyblue", width=1)

# ---- Attractor dengan Gaya Buatan ----
class Attractor:
    def __init__(self, x, y, canvas):
        self.mass = 20
        self.location = Vector(x, y)
        self.G = 0.6
        self.r = self.mass
        self.canvas = canvas
        self.oval = canvas.create_oval(x - self.r, y - self.r,
                                       x + self.r, y + self.r,
                                       fill="orange", outline="")

    def custom_force(self, m):
        force = self.location.subbed(m.location)
        distance = force.mag()
        distance = max(5.0, min(distance, 100.0))  # Clamp jarak
        force = force.normalized()

        # Gaya: tarik jika jauh, tolak jika dekat
        if distance < 60:
            strength = - (self.G * self.mass * m.mass) / (distance ** 2)
        else:
            strength = (self.G * self.mass * m.mass) / (distance ** 2)

        return force.multed(strength)

    def display(self):
        x, y, r = self.location.x, self.location.y, self.r
        self.canvas.coords(self.oval, x - r, y - r, x + r, y + r)

# ---- Setup Tkinter ----
WIDTH, HEIGHT = 800, 600
root = tk.Tk()
root.title("Exercise 2.9 - Gaya Buatan Sendiri")
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="black")
canvas.pack()

# Attractor di tengah
attractor = Attractor(WIDTH // 2, HEIGHT // 2, canvas)

# Banyak mover
movers = [Mover(random.uniform(1, 3),
                random.randint(0, WIDTH),
                random.randint(0, HEIGHT),
                canvas)
          for _ in range(20)]

# ---- Loop animasi ----
def draw():
    canvas.delete("trail")  # Hapus jejak saja (bukan oval)
    attractor.display()
    for mover in movers:
        force = attractor.custom_force(mover)
        mover.apply_force(force)
        mover.update()
        mover.display()
    root.after(33, draw)

draw()
root.mainloop()


In [ ]:
# SCRIPT 20 - Simulasi Gaya Tarik Gravitasi Sederhana
import tkinter as tk
import math
import time
from vector import Vector

# ------------------------
# Class Body (benda dengan massa, posisi, kecepatan, dll)
# ------------------------
class Body:
    def __init__(self, canvas, x, y, mass, color):
        self.canvas = canvas
        self.mass = mass
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.radius = math.sqrt(mass) * 2
        self.color = color
        self.shape = canvas.create_oval(x - self.radius, y - self.radius,
                                        x + self.radius, y + self.radius,
                                        fill=color)

    # Terapkan gaya ke benda (F = ma)
    def apply_force(self, force):
        a = force.dived(self.mass)
        self.acceleration.add(a)

    # Tarik benda lain dengan gaya gravitasi
    def attract(self, other):
        force = self.position.subbed(other.position)
        distance = max(5, min(force.mag(), 25))  # Hindari terlalu dekat atau jauh
        G = 1
        strength = (G * self.mass * other.mass) / (distance * distance)
        direction = force.normalized()
        attraction = direction.multed(strength)
        other.apply_force(attraction)
        return distance, strength  # Untuk ditampilkan di layar

    # Update posisi benda berdasarkan kecepatan dan percepatan
    def update(self):
        self.velocity.add(self.acceleration)
        self.position.add(self.velocity)
        self.acceleration = Vector(0, 0)  # Reset percepatan setelah update

    # Tampilkan benda di canvas
    def display(self):
        x = self.position.x
        y = self.position.y
        self.canvas.coords(self.shape,
                           x - self.radius, y - self.radius,
                           x + self.radius, y + self.radius)

# ------------------------
# Inisialisasi Tkinter
# ------------------------
WIDTH = 640
HEIGHT = 480
root = tk.Tk()
root.title("Simulasi Mutual Gravitational Attraction")
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg='white')
canvas.pack()

# Label untuk menampilkan info
info = tk.Label(root, text="", font=("Arial", 12))
info.pack()

# ------------------------
# Buat dua benda
# ------------------------
bodyA = Body(canvas, 200, 240, 20, 'red')
bodyB = Body(canvas, 440, 240, 20, 'blue')

# Kecepatan awal (agar mereka bergerak melingkar)
bodyA.velocity = Vector(0, -1.5)
bodyB.velocity = Vector(0, 1.5)

# ------------------------
# Fungsi animasi utama
# ------------------------
def animate():
    # Bersihkan layar dulu
    canvas.delete("vector")

    # Gaya tarik antara kedua benda
    d1, f1 = bodyA.attract(bodyB)
    d2, f2 = bodyB.attract(bodyA)

    # Update posisi dan gambar ulang
    bodyA.update()
    bodyB.update()
    bodyA.display()
    bodyB.display()

    # Tampilkan informasi gaya dan jarak
    info.config(text=f"Jarak: {d1:.2f} px | Gaya tarik: {f1:.4f} N\n"
                      f"Kecepatan A: ({bodyA.velocity.x:.2f}, {bodyA.velocity.y:.2f})\n"
                      f"Kecepatan B: ({bodyB.velocity.x:.2f}, {bodyB.velocity.y:.2f})")

    # Jalankan ulang animasi tiap 33 ms (30 FPS)
    root.after(33, animate)

# ------------------------
# Mulai animasi
# ------------------------
animate()
root.mainloop()


In [20]:
# SCRIPT 20 - Mutual Attraction Simulation 
import tkinter as tk
from vector import Vector
import random
import math

# ==== Mover Class ====
class Mover:
    def __init__(self, canvas, x, y, vx, vy, mass):
        self.canvas = canvas
        self.mass = mass
        self.position = Vector(x, y)
        self.velocity = Vector(vx, vy)
        self.acceleration = Vector(0, 0)
        self.radius = mass / 2
        self.body = canvas.create_oval(
            x - self.radius, y - self.radius,
            x + self.radius, y + self.radius,
            fill="skyblue"
        )

    def apply_force(self, force):
        f = force.dived(self.mass)  # F = m * a -> a = F / m
        self.acceleration.add(f)

    def attract(self, other):
        G = 0.4  # konstanta gravitasi
        force = self.position.subbed(other.position)
        distance = force.mag()
        distance = max(5.0, min(distance, 25.0))  # batas bawah dan atas jarak
        strength = G * self.mass * other.mass / (distance * distance)
        force = force.normalized().multed(strength)
        other.apply_force(force)

    def update(self):
        self.velocity.add(self.acceleration)
        self.position.add(self.velocity)
        self.acceleration = Vector(0, 0)

    def display(self):
        x = self.position.x
        y = self.position.y
        r = self.radius
        self.canvas.coords(self.body, x - r, y - r, x + r, y + r)

# ==== Setup Tkinter ====
WIDTH, HEIGHT = 800, 600
root = tk.Tk()
root.title("Mutual Attraction Simulation")
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="black")
canvas.pack()

# ==== Buat Matahari (Sun) di tengah ====
sun = Mover(canvas, WIDTH // 2, HEIGHT // 2, 0, 0, 100)
sun.canvas.itemconfig(sun.body, fill="orange")

# ==== Buat banyak mover mengorbit sun ====
movers = []
for _ in range(30):
    angle = random.uniform(0, 2 * math.pi)
    distance = random.uniform(100, 200)
    x = WIDTH // 2 + math.cos(angle) * distance
    y = HEIGHT // 2 + math.sin(angle) * distance
    vx = -math.sin(angle) * random.uniform(1, 2)
    vy = math.cos(angle) * random.uniform(1, 2)
    mass = random.uniform(5, 10)
    movers.append(Mover(canvas, x, y, vx, vy, mass))

# ==== Loop animasi ====
def animate():
    for mover in movers:
        sun.attract(mover)  # sun menarik semua mover
        for other in movers:
            if mover != other:
                mover.attract(other)  # saling tarik antar mover
        mover.update()
        mover.display()
    root.after(33, animate)

animate()
root.mainloop()

In [ ]:
# SCRIPT  - 


In [ ]:
# SCRIPT  - 


In [ ]:
# SCRIPT  - 


In [ ]:
# SCRIPT  - 


In [ ]:
# SCRIPT  - 


In [ ]:
# SCRIPT  - 


In [ ]:
# SCRIPT  - 


In [ ]:
# SCRIPT  - 


In [ ]:
# SCRIPT  - 


In [ ]:
# SCRIPT  - 


In [ ]:
# SCRIPT  - 


In [ ]:
#SCRIPT  - 
